In [2]:
!pip install pyspark pandas

  Using cached pyspark-3.5.2.tar.gz (317.3 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=2e4780f4b4da146fd0f614d4a6979b19d19e3e37fc21c6303db8ea8325bbbfaa
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
# Import necessary libraries
from pyspark.sql import SparkSession
# Initialize Spark session
spark = SparkSession.builder.appName("HousePricePrediction").getOrCreate()

In [4]:
# Step 1: Load Data
# Sample dataset
data = [
 (1200, 3, 2, 300000),
 (1500, 4, 3, 350000),
 (800, 2, 1, 200000),
 (2500, None, 3, 450000),
 (1800, 4, 2, 400000),
 (2500, 5, 3, 500000),
 (None, 4, 3, 400000)
]
# Define schema
columns = ["SquareFootage", "Bedrooms", "Bathrooms",
"Price"]

In [5]:
# Create DataFrame
df = spark.createDataFrame(data, schema=columns)
df.show()

+-------------+--------+---------+------+
|SquareFootage|Bedrooms|Bathrooms| Price|
+-------------+--------+---------+------+
|         1200|       3|        2|300000|
|         1500|       4|        3|350000|
|          800|       2|        1|200000|
|         2500|    NULL|        3|450000|
|         1800|       4|        2|400000|
|         2500|       5|        3|500000|
|         NULL|       4|        3|400000|
+-------------+--------+---------+------+



In [6]:
# Step 2: Data Preprocessing
from pyspark.ml.feature import VectorAssembler, StandardScaler
# Check for missing values
df.describe().show()


+-------+------------------+------------------+------------------+-----------------+
|summary|     SquareFootage|          Bedrooms|         Bathrooms|            Price|
+-------+------------------+------------------+------------------+-----------------+
|  count|                 6|                 6|                 7|                7|
|   mean|1716.6666666666667|3.6666666666666665|2.4285714285714284|371428.5714285714|
| stddev| 691.1343333004565|1.0327955589886444|0.7867957924694432| 99402.9797388005|
|    min|               800|                 2|                 1|           200000|
|    max|              2500|                 5|                 3|           500000|
+-------+------------------+------------------+------------------+-----------------+



In [7]:
# Fill missing values (if any) with mean of the column
df = df.na.fill({
 'SquareFootage': df.agg({'SquareFootage': 'mean'}).collect()[0][0],
 'Bedrooms': df.agg({'Bedrooms': 'mean'}).collect()[0][0],
 'Bathrooms': df.agg({'Bathrooms': 'mean'}).collect()[0][0]
})
df.show()


+-------------+--------+---------+------+
|SquareFootage|Bedrooms|Bathrooms| Price|
+-------------+--------+---------+------+
|         1200|       3|        2|300000|
|         1500|       4|        3|350000|
|          800|       2|        1|200000|
|         2500|       3|        3|450000|
|         1800|       4|        2|400000|
|         2500|       5|        3|500000|
|         1716|       4|        3|400000|
+-------------+--------+---------+------+



In [8]:
# Normalize the features using StandardScaler if needed:
# Assemble features into a vector
assembler = VectorAssembler(inputCols=["SquareFootage", "Bedrooms",
"Bathrooms"], outputCol="unscaled_features")
output = assembler.transform(df)
# Scale features
scaler = StandardScaler(inputCol="unscaled_features", outputCol="features",
withStd=True, withMean=True)
scaler_model = scaler.fit(output)
scaled_output = scaler_model.transform(output)
# Show scaled features
scaled_output.select("features", "Price").show()

+--------------------+------+
|            features| Price|
+--------------------+------+
|[-0.8187635537508...|300000|
|[-0.3432648084862...|350000|
|[-1.4527618807703...|200000|
|[1.24173100906247...|450000|
|[0.13223393677835...|400000|
|[1.24173100906247...|500000|
|[-9.0571189574193...|400000|
+--------------------+------+



In [9]:
# Split the data into training and test sets
train_data, test_data = scaled_output.randomSplit([0.8, 0.2])
# Step 3: Model Training
from pyspark.ml.regression import LinearRegression
# Initialize the linear regression model
lr = LinearRegression(labelCol="Price", featuresCol="features")
# Fit the model to the training data
model = lr.fit(train_data)

In [10]:
# Step 4: Model Evaluation
from pyspark.ml.evaluation import RegressionEvaluator
# Make predictions on the test data
predictions = model.transform(test_data)
predictions.select("features", "Price", "prediction").show()
# Evaluate the model's performance
evaluator = RegressionEvaluator(labelCol="Price", predictionCol="prediction",
metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")
# Make predictions on the test data
predictions = model.transform(test_data)
# Calculate R-squared
evaluator_r2 = RegressionEvaluator(labelCol="Price", predictionCol="prediction",
metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print(f"R-squared (R²): {r2}")
# Calculate Mean Absolute Error (MAE)
evaluator_mae = RegressionEvaluator(
 labelCol="Price", predictionCol="prediction", metricName="mae"
)
mae = evaluator_mae.evaluate(predictions)
print(f"Mean Absolute Error (MAE): {mae}")

+--------------------+------+-----------------+
|            features| Price|       prediction|
+--------------------+------+-----------------+
|[-1.4527618807703...|200000|392857.1428571429|
|[-0.8187635537508...|300000|402380.9523809524|
|[-0.3432648084862...|350000|409523.8095238096|
|[-9.0571189574193...|400000|414666.6666666667|
|[1.24173100906247...|500000|416666.6666666667|
+--------------------+------+-----------------+

Root Mean Squared Error (RMSE): 108054.03871881291
R-squared (R²): -0.16756752834467203
Mean Absolute Error (MAE): 90552.38095238098


In [13]:
# Save the model for future use
model.write().overwrite().save("house_price_model")
# PREDICT Price for given data (SquareFootage:1600, Bedrooms:3,Bathrooms:2)
from pyspark.ml.regression import LinearRegressionModel
# Load the saved model
model = LinearRegressionModel.load("house_price_model")
# New data example (SquareFootage, Bedrooms, Bathrooms)
new_df = spark.createDataFrame([(1600, 3, 2)], schema=["SquareFootage",
"Bedrooms", "Bathrooms"])
# Assemble features into a single vector (no need to scale if the model was trained on unscaled data)
assembler = VectorAssembler(inputCols=["SquareFootage", "Bedrooms",
"Bathrooms"], outputCol="features")
new_vector_data = assembler.transform(new_df)
# Make prediction
new_predictions = model.transform(new_vector_data)
# Show the prediction
new_predictions.show()


+-------------+--------+---------+----------------+--------------------+
|SquareFootage|Bedrooms|Bathrooms|        features|          prediction|
+-------------+--------+---------+----------------+--------------------+
|         1600|       3|        2|[1600.0,3.0,2.0]|2.4424644798391443E7|
+-------------+--------+---------+----------------+--------------------+

